In [4]:
from my_package.summary_evalution import *
import pandas as pd

In [5]:
def calculate_rouge_scores_df(reference_df, system_df, reference_column='Original Summary', system_column=None):
    if system_column is None:
        system_column = system_df.columns[0]
    
    rouge = ROUGEScore()
    rouge_scores = []
    for i in range(len(reference_df)):
        reference_summary = reference_df[reference_column][i]
        system_summary = system_df[system_column][i]
        scores = rouge(reference_summary, system_summary)
        rouge_scores.append(scores)
    return pd.DataFrame(rouge_scores)

In [49]:
ref = "4th" 
fileName = r"DUC_Dynamic-Sent_.csv"
evalFolder = r"DUC/Algorithm Eval Scores/Dynamic/"+ref+" Ref/"
df = pd.read_csv(fileName)

# Change Golden Summary to another reference

In [50]:
refL = ref
if(refL == "1st"):
    refL = "first"
elif(refL == "2nd"):
    refL = "second"
elif(refL == "3rd"):
    refL = "third"
elif(refL == "4th"):
    refL = "fourth"
else:
    print("Error Ref")
refDf = pd.read_csv(r"DUC/duc_main_datasets/Duc_dataset_"+refL+"_ref_summary.csv")
df = df.drop(columns=['Original Summary'])
df.insert(1, 'Original Summary',  refDf['Original Summary'])
df.head()

,Original Article,Original Summary,"('Tm',)","('Lex',)","('Luhn',)","('Lsa',)","('Tr',)","('LDA',)","('Tm', 'Lex')","('Tm', 'Luhn')",...,"('Lex', 'Luhn', 'Tr', 'LDA')","('Lex', 'Lsa', 'Tr', 'LDA')","('Luhn', 'Lsa', 'Tr', 'LDA')","('Tm', 'Lex', 'Luhn', 'Lsa', 'Tr')","('Tm', 'Lex', 'Luhn', 'Lsa', 'LDA')","('Tm', 'Lex', 'Luhn', 'Tr', 'LDA')","('Tm', 'Lex', 'Lsa', 'Tr', 'LDA')","('Tm', 'Luhn', 'Lsa', 'Tr', 'LDA')","('Lex', 'Luhn', 'Lsa', 'Tr', 'LDA')","('Tm', 'Lex', 'Luhn', 'Lsa', 'Tr', 'LDA')"
0,\r\nCambodian leader Hun Sen on Friday rejecte...,"Cambodian elections, fraudulent according to o...",last week hun sen s cambodian people s party a...,government and opposition parties have asked k...,last week hun sen s cambodian people s party a...,the cambodian people s party criticized a non ...,hun sen s party won of the seats in parliament...,the deal which will make hun sen prime ministe...,last week hun sen s cambodian people s party a...,last week hun sen s cambodian people s party a...,...,last week hun sen s cambodian people s party a...,the deal which will make hun sen prime ministe...,last week hun sen s cambodian people s party a...,last week hun sen s cambodian people s party a...,last week hun sen s cambodian people s party a...,last week hun sen s cambodian people s party a...,the deal which will make hun sen prime ministe...,last week hun sen s cambodian people s party a...,last week hun sen s cambodian people s party a...,last week hun sen s cambodian people s party a...
1,\r\nHonduras braced for potential catastrophe ...,"A category 5 storm, Hurricane Mitch roared acr...",at least people have been confirmed dead in ho...,in belize a hurricane watch was in place and t...,in la ceiba on honduras northern coast people ...,with the storm seemingly anchored off honduras...,million in relief to honduras nicaragua el sal...,the honduran president closed schools and publ...,at least people have been confirmed dead in ho...,the honduran president closed schools and publ...,...,in washington on thursday president bill clint...,in washington on thursday president bill clint...,in washington on thursday president bill clint...,in washington on thursday president bill clint...,the honduran president closed schools and publ...,in washington on thursday president bill clint...,in washington on thursday president bill clint...,in washington on thursday president bill clint...,in washington on thursday president bill clint...,in washington on thursday president bill clint...
2,\r\nCuban President Fidel Castro said Sunday h...,"On Oct. 16, 1998 British police arrested forme...",the judge is probing pinochet s role in the de...,in london where pinochet has been under arrest...,pinochet was placed under arrest in london fri...,the tone of the british prime minister s first...,at the instigation of a spanish magistrate see...,pinochet who has vowed to fight attempts to ex...,the judge is probing pinochet s role in the de...,the judge is probing pinochet s role in the de...,...,pinochet was detained by british police in lon...,pinochet who has vowed to fight attempts to ex...,pinochet was arrested in london on oct. at the...,the judge is probing pinochet s role in the de...,british police arrested pinochet in his bed fr...,pinochet was detained by british police in lon...,pinochet who has vowed to fight attempts to ex...,pinochet was detained by british police in lon...,pinochet was detained by british police in lon...,pinochet was detained by british police in lon...
3,"\r\nMUNICH, Germany (AP) _ U.S. prosecutors ha...",Following the bombings of the embassies in Afr...,saudi exile osama bin laden the alleged master...,administration officials have denied since aug...,bin laden is believed to be living in afghanis...,officials say u.s. intelligence has received r...,at the time of the aug. cruise missile attack ...,saudi exile osama bin laden the alleged master...,saudi exile osama bin laden the alleged master...,police seized copies of th

# Main Run

In [51]:
frames = []
algos = df.columns[2:]
for i in tqdm(range(len(algos))):
    key = algos[i]
    summdf = pd.DataFrame([df[key]])
    summdf = summdf.T
    temp = calculate_rouge_scores_df(df,summdf)
    temp.to_csv(evalFolder + "Eval_Score_" + str(key)+".csv")
    temp = calculate_average_by_column_df(temp)
    temp = pd.DataFrame(temp).T
    temp["Algorithm"] = key
    frames.append(temp)
out = pd.concat(frames)
cols = list(out.columns)
cols = [cols[-1]] + cols[:-1]
out = out[cols]
out = out.T
out.columns = out.iloc[0]
out = out[1:]
out.to_csv(fileName[:-4]+""+ref+"_Score.csv", index=False)

100%|██████████| 63/63 [05:14<00:00,  4.99s/it]


# Add Metric Column

In [16]:
path = r"BBC_6Sent_Score.csv"
df = pd.read_csv(path)
df2 = pd.read_csv(r"DUC/DUC_by_length(5,6,7)/Duc_third_reference_scores_by_length/5.csv")
df.insert(loc=0, column="Metric", value=df2["Unnamed: 0"])
df.to_csv(path, index=False)

In [15]:
df

,Metric,"('Tm',)","('Lex',)","('Luhn',)","('Lsa',)","('Tr',)","('LDA',)","('Tm', 'Lex')","('Tm', 'Luhn')","('Tm', 'Lsa')",...,"('Lex', 'Luhn', 'Tr', 'LDA')","('Lex', 'Lsa', 'Tr', 'LDA')","('Luhn', 'Lsa', 'Tr', 'LDA')","('Tm', 'Lex', 'Luhn', 'Lsa', 'Tr')","('Tm', 'Lex', 'Luhn', 'Lsa', 'LDA')","('Tm', 'Lex', 'Luhn', 'Tr', 'LDA')","('Tm', 'Lex', 'Lsa', 'Tr', 'LDA')","('Tm', 'Luhn', 'Lsa', 'Tr', 'LDA')","('Lex', 'Luhn', 'Lsa', 'Tr', 'LDA')","('Tm', 'Lex', 'Luhn', 'Lsa', 'Tr', 'LDA')"
0,rouge1_fmeasure,0.391201,0.385570,0.382211,0.367823,0.264301,0.369146,0.400364,0.408247,0.395122,...,0.374198,0.360130,0.386150,0.404775,0.421186,0.400729,0.387999,0.402670,0.394603,0.405679
1,rouge1_precision,0.280347,0.273275,0.263052,0.269877,0.177831,0.271063,0.287556,0.287778,0.288458,...,0.262193,0.259858,0.275924,0.289665,0.304156,0.284548,0.281485,0.289377,0.282293,0.291796
2,rouge1_recall,0.798079,0.818432,0.892452,0.694832,0.699016,0.695746,0.811768,0.877792,0.764292,...,0.825506,0.721524,0.798353,0.828683,0.840760,0.841183,0.761868,0.816664,0.810179,0.819657
3,rouge2_fmeasure,0.324752,0.323417,0.338944,0.281901,0.191960,0.282482,0.336738,0.360470,0.321127,...,0.317467,0.279852,0.323302,0.346201,0.364414,0.345414,0.314042,0.342095,0.332618,0.345062
4,rouge2_precision,0.231620,0.227900,0.232042,0.206572,0.129143,0.206794,0.240699,0.252457,0.233416,...,0.221246,0.201045,0.229778,0.246225,0.261399,0.243832,0.226744,0.244401,0.236517,0.246725
5,rouge2_recall,0.674892,0.703756,0.809046,0.535090,0.514607,0.537237,0.694587,0.794353,0.631545,...,0.716266,0.571559,0.684218,0.725438,0.744628,0.741603,0.628589,0.710235,0.699020,0.713115
6,rougeL_fmeasure,0.317375,0.316664,0.321694,0.289296,0.211761,0.291485,0.326975,0.339702,0.316933,...,0.307721,0.286702,0.316104,0.333050,0.347433,0.333184,0.311623,0.329340,0.323420,0.331808
7,rougeL_precision,0.224238,0.221155,0.218597,0.209422,0.140947,0.211098,0.231340,0.236042,0.228118,...,0.212653,0.203903,0.222936,0.235086,0.247229,0.233519,0.222817,0.233428,0.228218,0.235323
8,rougeL_recall,0.672433,0.700621,0.781435,0.565089,0.583201,0.568220,0.688986,0.760545,0.635712,...,0.707195,0.596973,0.679340,0.708466,0.720685,0.725234,0.635626,0.693966,0.690079,0.697080
9,rougeLsum_fmeasure,0.363484,0.358389,0.361574,0.333555,0.239955,0.334333,0.373118,0.385591,0.363594,...,0.349056,0.326876,0.358170,0.378889,0.395553,0.376030,0.356201,0.375434,0.366811,0.378244
